In [13]:
import pandas as pd
import json
from pprint import pformat
import sqlite3

In [17]:
def get_set_from_json(file_name:str):
    with open(file_name) as f:
        data = json.load(f)

    lotr_set = data['data']['cards']

    return lotr_set

lotr_set = get_set_from_json('ltr.json')

In [19]:
drop_list = ['artist','availability','borderColor','colors','colorIdentity',
    'convertedManaCost','edhrecRank','colorIdentity','foreignData',
    'frameVersion','isStarter','language','layout','legalities',
    'manaCost','manaValue','printings','securityStamp','setCode',
    'power', 'toughness', 'edhrecSaltiness','isReprint','flavorName',
    'hasAlternativeDeckLimit']

all_keys = set().union(*(d.keys() for d in lotr_set))
all_keys = sorted(all_keys)
new_dict = {}
lists_keys = []
dicts_keys = []
for key in all_keys:
    new_dict[key] = [card[key] if key in card else None for card in lotr_set]
for key in drop_list:
    if key in new_dict:
        new_dict.pop(key)
for key in new_dict:
    for i in range(len(new_dict[key])):
        if new_dict[key][i] == None:
            continue
        elif isinstance(new_dict[key][i], list):
            lists_keys.append(key)
            break
        elif isinstance(new_dict[key][i], dict):
            dicts_keys.append(key)
            break

for key in lists_keys:
    new_dict.pop(key)
for key in dicts_keys:
    new_dict.pop(key)

lists_keys.append('uuid')
lists_keys.append('number')

print('New Dict',new_dict.keys())
print('Lists',lists_keys)
print('dicts',dicts_keys)

New Dict dict_keys(['asciiName', 'flavorText', 'hasFoil', 'hasNonFoil', 'isAlternative', 'isFullArt', 'isPromo', 'name', 'number', 'rarity', 'text', 'type', 'uuid'])
Lists ['boosterTypes', 'finishes', 'frameEffects', 'keywords', 'promoTypes', 'subtypes', 'supertypes', 'types', 'variations', 'uuid', 'number']
dicts ['identifiers', 'leadershipSkills', 'purchaseUrls']


In [31]:
# Transforming the dictionary into a dataframe
df = pd.DataFrame.from_dict(new_dict)
df.set_index('uuid', inplace=True)
df.sort_values(by=['number'])
df = df.dropna(axis=1, how='all')
df = df.dropna(axis=0, how='all')
df = df.fillna(0)


In [ ]:
# save to ltr.db
conn = sqlite3.connect('ltr.db')
df.to_sql('cards', conn, if_exists='replace')
conn.close()